<a href="https://colab.research.google.com/github/Chiamakac/CARE/blob/main/CANADA/CARE_discoveries.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!git clone https://github.com/Chiamakac/CARE.git

Cloning into 'CARE'...
remote: Enumerating objects: 99, done.
remote: Total 99 (delta 0), reused 0 (delta 0), pack-reused 99
Unpacking objects: 100% (99/99), 41.35 MiB | 9.63 MiB/s, done.


In [3]:
pip install -q pdfminer.six

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 47.7 MB/s eta 0:00:00


# **TEXT EXTRACTION**
This extracts the font name, font_size, text, COLOR page by page 

In [4]:
from pdfminer.high_level import extract_pages        # Import function for extracting PDF pages
from pdfminer.layout import LTTextContainer, LTChar, LTLine, LAParams   # Import PDF layout classes
import os   # Import operating system functions

path = r'/content/CARE/sample_Eng.pdf'    # Define path to PDF file

output_dir = 'Dextracts'   # Define directory for output files
os.makedirs(output_dir, exist_ok=True)   # Create output directory if it doesn't exist

for page_number, page_layout in enumerate(extract_pages(path), start=1):     # Iterate over PDF pages
    output_file = os.path.join(output_dir, f'page{page_number}_info.txt')   # Define name of output file for current page
    with open(output_file, 'w') as file:     # Open the output file for writing
        for element in page_layout:     # Iterate over elements on each page
            if isinstance(element, LTTextContainer):   # Check if element is a text container
                for text_line in element:   # Iterate over lines of text in the container
                    if isinstance(text_line, LTTextContainer):   # Check if line is a text container
                        if len(text_line._objs) > 0:    # Check if line contains any objects
                            first_char = text_line._objs[0]   # Get the first object in the line
                            if isinstance(first_char, LTChar):   # Check if object is a character
                                font_size = first_char.size   # Get the font size of the character
                                font_name = first_char.fontname   # Get the font name of the character
                                font_color = first_char.graphicstate.ncolor   # Get the font color of the character
                                text = element.get_text()   # Get the text of the element
                                file.write(f"Font size: {font_size}\n")   # Write font size to output file
                                file.write(f"Font name: {font_name}\n")   # Write font name to output file
                                file.write(f"Font color: {font_color}\n")   # Write font color to output file
                                file.write(f"Text: {text}")   # Write text to output file
                                file.write("\n")   # Add newline to separate output for different elements
                                # print(f"Font color: {font_color}  Text: {text}")  # Display font color and text on console


## Majesty `begin code`

In [ ]:
#import pandas, we will need it

In [1]:
import pandas as pd

Get the list of `pages` from the extracted pdf file stored in the `Dextracts` folder

In [5]:
file_name =  path.split('/')[-1]
output_folder = 'Dextracts'
pages = sorted(os.listdir(output_folder))

Define the `get_font_detail_text(sec)` function that extracts the texts and details from a section 

In [6]:
def get_font_detail_text(sec):
  font_detail=text=''
  try:
    font_detail, text = sec.split('Text: ')
    text, (f_size, f_name, f_color) = text.replace('\n', ' '), [
      item.rsplit(': ', 1)[1] for item in font_detail.strip().splitlines()]
  except:
    pass
  if text: return text, f_size, f_name, f_color

Using the above function, create a list of section details for each section in the page

In [7]:
section_details=[]
for page in pages[2:3]:
  page_num = page.split('_')[0]
  section_details = [(file_name, len(pages), page_num, 'English')+get_font_detail_text(section) for section in 
                     open(f"Dextracts/{page}").read().split("\n\n")
                     if get_font_detail_text(section)]

Remove the repeated sections by ensuring that only one unique content is retained

In [8]:
section_details = {details[4]:details for details in section_details}.values()

Unzip the list details into a tuple of lists and create a dictionary `section_dict` using each of the required column heads as the key with the corresponding list of value 

In [9]:
section_dict={}
file_names, num_pages, page_num, language, content, fsize, fname, fcolor = list(zip(*section_details))
section_dict['Font Filename'] = file_names
section_dict['Number of pages'] = num_pages
section_dict['Language'] = language
section_dict['Content Type'] = ['Section Title' if int(float(s))>10 else 'Running Text' for s in fsize]
section_dict['Content'] = content
section_dict['Start Page'] = page_num
section_dict['End Page'] = page_num
section_dict['Font Size'] = fsize
section_dict['Font Name'] = fname
section_dict['Font Colour'] = fcolor

Convert the `section_dict` to a `pandas` dataframe

In [10]:
report = pd.DataFrame.from_dict(section_dict)

View report dataframe

In [11]:
report

,Font Filename,Number of pages,Language,Content Type,Content,Start Page,End Page,Font Size,Font Name,Font Colour
0,sample_Eng.pdf,8,English,Section Title,REPORT TO SHAREHOLDERS,page3,page3,11.0,NDJLVI+ArialNarrow-Bold,"(0.75, 0.05, 1, 0)"
1,sample_Eng.pdf,8,English,Running Text,2018 marked a change in food and even ...,page3,page3,9.0,AMVLVI+ArialNarrow,"(0, 0, 0, 1)"
2,sample_Eng.pdf,8,English,Running Text,"Overall, with the challenging rigid packaging ...",page3,page3,9.0,AMVLVI+ArialNarrow,"(0, 0, 0, 1)"
3,sample_Eng.pdf,8,English,Running Text,"The continued progression at the Winnipeg, Man...",page3,page3,9.0,FRIPNY+ArialNarrow,"(0, 0, 0, 1)"
4,sample_Eng.pdf,8,English,Running Text,"Similarly, Winpak’s specialty films business i...",page3,page3,9.0,AMVLVI+ArialNarrow,"(0, 0, 0, 1)"
5,sample_Eng.pdf,8,English,Running Text,2018 was again remarkable for the Winpak–Sojit...,page3,page3,9.0,AMVLVI+ArialNarrow,"(0, 0, 0, 1)"
6,sample_Eng.pdf,8,English,Running Text,"Overall, Winpak’s flexible packaging business ...",page3,page3,9.0,FRIPNY+ArialNarrow,"(0, 0, 0, 1)"
7,sample_Eng.pdf,8,English,Running Text,"In 2018, the Company’s rigid container busines...",page3,page3,9.0,AMVLVI+ArialNarrow,"(0, 0, 0, 1)"
8,sample_Eng.pdf,8,English,Running Text,The Company’s product offering as a system of ...,page3,page3,9.0,AMVLVI+ArialNarrow,"(0, 0, 0, 1)"
9,sample_Eng.pdf,8,English,Running Text,Winpak’s packaging machinery division in ...,page3,page3,9.0,AMVLVI+ArialNarrow,"(0, 0, 0, 1)"


Finally, write to an Excel file which should be similar to the CSV but not exactly the same...

In [ ]:
with pd.ExcelWriter('AnnualReports.xlsx') as writer:  
    report.to_excel(writer, sheet_name='Files Details')

...or you can write to a CSV file

In [ ]:
report.to_csv('AnnualReports.csv') 

If you want to save the output to another folder instead of the working current woring folder...

In [ ]:
from pathlib import Path  
filepath = Path('output_dir/AnnualReports.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True)  
report.to_csv(filepath)

* Now, you can view either the csv in colab or download the xlsx to open in Excel. 
* With a little tweaking you can wrap the whole process inside a loop and use it to read and process multiple similar files
* I feel that the Excel format may be a good way to present the output from multiple files in different worksheets
* Finally, I tried to include some explanations but if you have further questions ask me or ChatGPT

## Majesty: `end code`

# **FOR IMAGE EXTRACTION**

In [ ]:
pip install -qqq PyMuPDF

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 56.1 MB/s eta 0:00:00


In [ ]:
pip install -qqq pillow

In [ ]:
# import libraries
import fitz
import io
from PIL import Image, ImageDraw

In [ ]:
import os
import io
from PIL import Image, ImageDraw
import fitz

# file path you want to extract images from
file = "/content/CARE/sample_Eng.pdf"

# create output directory if it doesn't exist
output_dir = "output_images_fr"
if not os.path.exists(output_dir):
    os.mkdir(output_dir)

# open the file
pdf_file = fitz.open(file)

# iterate over PDF pages
for page_index in range(len(pdf_file)):
    # get the page itself
    page = pdf_file[page_index]
    
    # get a list of images on the page
    image_list = page.get_images()

    # # printing number of images found in this page
    # if image_list:
    #     print(f"[+] Found a total of {len(image_list)} images in page {page_index}")
    # else:
    #     print("[!] No images found on page", page_index)
    
    # iterate over each image on the page
    for image_index, img in enumerate(image_list, start=1):
        # get the XREF of the image
        xref = img[0]

        # extract the image bytes
        image_bytes = pdf_file.extract_image(xref)["image"]

        # create a PIL image from the bytes
        pil_image = Image.open(io.BytesIO(image_bytes))

        # draw image and save to a file
        draw = ImageDraw.Draw(pil_image)
        output_file = os.path.join(output_dir, f"page_{page_index}_image_{image_index}.png")
        pil_image.save(output_file, "PNG")
        
        # print(f"[+] Saved image {image_index} on page {page_index} to {output_file}")


# **TABLE EXTRACTION**

In [ ]:
pip install -qqq PyPDF2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 5.7 MB/s eta 0:00:00


In [ ]:
pip install -qqq tabula-py

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 61.0 MB/s eta 0:00:00


In [ ]:
import os
import tabula

# Open the PDF file
with open('/content/CARE/sample_Eng.pdf', 'rb') as pdf_file:

    # Extract tables from the PDF file
    tables = tabula.read_pdf(pdf_file, pages='all')
    
    # Create a folder to store the Excel files
    folder_name = 'tables_fr'
    if not os.path.isdir(folder_name):
        os.mkdir(folder_name)
    
    # Iterate over extracted tables and export as Excel individually
    for i, table in enumerate(tables, start=1):
        table.to_excel(os.path.join(folder_name, f"table_{i}.xlsx"), index=False)



Obtain the location of text(**Language free**) in a page using **pdfminer**

In [ ]:
# Import necessary packages
from pdfminer.pdfdocument import PDFDocument
from pdfminer.pdfpage import PDFPage
from pdfminer.pdfparser import PDFParser
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import PDFPageAggregator, TextConverter
from pdfminer.layout import LAParams, LTTextBox, LTTextLine, LTFigure
from io import StringIO

# Define function to recursively parse the layout tree
def parse_layout(layout, outfp):
    """
    Recursively parse the layout tree and write information
    about each object's class name, bounding box, and text (if any) to the output file.
    """
    for lt_obj in layout:
        # Write the class name and bounding box of the object to the output file
        # outfp.write(lt_obj.__class__.__name__ + "\n")
        # outfp.write(str(lt_obj.bbox) + "\n")

        # If the object is a text box or line, write its text to the output file
        if isinstance(lt_obj, LTTextBox) or isinstance(lt_obj, LTTextLine):
            outfp.write(lt_obj.get_text() + "\n")

        # If the object is a figure, recursively parse its layout
        elif isinstance(lt_obj, LTFigure):
            parse_layout(lt_obj, outfp)

# Open the PDF file in binary mode
fp = open('/content/CARE/sample_Eng.pdf', 'rb')

# Create a PDF parser object from the binary file
parser = PDFParser(fp)

# Create a PDF document object from the parser object
doc = PDFDocument(parser)

# Set up the PDFResourceManager, LAParams, and PDFPageAggregator objects

# Create a PDF resource manager object
rsrcmgr = PDFResourceManager()

# Set the layout parameters for the PDF document
laparams = LAParams()

# Create a PDF device object to extract layout information from each page
device = PDFPageAggregator(rsrcmgr, laparams=laparams)

# Create a PDF interpreter object to process the page contents
interpreter = PDFPageInterpreter(rsrcmgr, device)

# Open the output file
with open('Foutput4.txt', 'w', encoding='utf-8') as outfp:
    # Loop through each page of the PDF document and process the page contents using the PDF interpreter
    for page in PDFPage.create_pages(doc):
        interpreter.process_page(page)
        layout = device.get_result()

        # Parse the layout tree for this page and write the information to the output file
        parse_layout(layout, outfp)


In [ ]:
# This extracts the font name, font_size, text, page by page

# from pdfminer.high_level import extract_pages        # Import function for extracting PDF pages
# from pdfminer.layout import LTTextContainer, LTChar, LTLine, LAParams   # Import PDF layout classes
# import os   # Import operating system functions

# path = r'/content/CARE/sample_Eng.pdf'    # Define path to PDF file

# output_dir = 'extracts'   # Define directory for output files
# os.makedirs(output_dir, exist_ok=True)   # Create output directory if it doesn't exist

# for page_number, page_layout in enumerate(extract_pages(path), start=1):     # Iterate over PDF pages
#     output_file = os.path.join(output_dir, f'page{page_number}_font_info.txt')   # Define name of output file for current page
#     with open(output_file, 'w') as file:     # Open the output file for writing
#         for element in page_layout:     # Iterate over elements on each page
#             if isinstance(element, LTTextContainer):   # Check if element is a text container
#                 for text_line in element:   # Iterate over lines of text in the container
#                     if isinstance(text_line, LTTextContainer):   # Check if line is a text container
#                         if len(text_line._objs) > 0:    # Check if line contains any objects
#                             first_char = text_line._objs[0]   # Get the first object in the line
#                             if isinstance(first_char, LTChar):   # Check if object is a character
#                                 font_size = first_char.size   # Get the font size of the character
#                                 font_name = first_char.fontname   # Get the font name of the character
#                                 text = element.get_text()   # Get the text of the element
#                                 file.write(f"Font size: {font_size}\n")   # Write font size to output file
#                                 file.write(f"Font name: {font_name}\n")   # Write font name to output file
#                                 file.write(f"Text: {text}")   # Write text to output file
#                                 file.write("\n")   # Add newline to separate output for different elements


In [ ]:
# from pdfminer.high_level import extract_pages        # Import function for extracting PDF pages
# from pdfminer.layout import LTTextContainer, LTChar, LTLine, LAParams   # Import PDF layout classes
# import os   # Import operating system functions

# path = r'/content/CBBD_Annual_Report_2017_FR.pdf'    # Define path to PDF file

# output_file = 'SEnglish_font_info.txt'   # Define name of output file

# with open(output_file, 'w') as file:     # Open the output file for writing
#     for page_layout in extract_pages(path):     # Iterate over PDF pages
#         for element in page_layout:     # Iterate over elements on each page
#             if isinstance(element, LTTextContainer):   # Check if element is a text container
#                 for text_line in element:   # Iterate over lines of text in the container
#                     if isinstance(text_line, LTTextContainer):   # Check if line is a text container
#                         if len(text_line._objs) > 0:    # Check if line contains any objects
#                             first_char = text_line._objs[0]   # Get the first object in the line
#                             if isinstance(first_char, LTChar):   # Check if object is a character
#                                 font_size = first_char.size   # Get the font size of the character
#                                 font_name = first_char.fontname   # Get the font name of the character
#                                 text = element.get_text()   # Get the text of the element
#                                 file.write(f"Font size: {font_size}\n")   # Write font size to output file
#                                 file.write(f"Font name: {font_name}\n")   # Write font name to output file
#                                 file.write(f"Text: {text}")   # Write text to output file
#                                 file.write("\n")   # Add newline to separate output for different elements


Code to count the **font size** frquencies

In [ ]:
pip install -q PyPDF2

In [ ]:
import PyPDF2

In [ ]:
from pathlib import Path  # import the Path class from the pathlib module
from typing import Iterable, Any  # import Iterable and Any from the typing module
from pdfminer.high_level import extract_pages  # import the extract_pages function from pdfminer.high_level

def get_indented_name(o: Any, depth: int) -> str:
    """Indented name of class"""
    return '  ' * depth + o.__class__.__name__  # return the indented class name based on depth

def get_optional_fontinfo(o: Any) -> str:
    """Font info of LTChar if available, otherwise empty string"""
    if hasattr(o, 'fontname') and hasattr(o, 'size'):  # check if 'fontname' and 'size' attributes exist in the object
        return f'{o.fontname} {round(o.size)}pt'  # if yes, return the font name and size as a string
    return ''  # if not, return an empty string

def get_optional_text(o: Any) -> str:
    """Text of LTItem if available, otherwise empty string"""
    if hasattr(o, 'get_text'):  # check if the object has a 'get_text' method
        return o.get_text().strip()  # if yes, return the text obtained from the method call, stripped of leading/trailing white space
    return ''  # if not, return an empty string

def count_font_sizes(o: Any, size_dict: dict):
    """Count the frequency of font sizes"""
    if isinstance(o, Iterable):  # check if the object is iterable
        for i in o:  # if yes, iterate over its elements
            count_font_sizes(i, size_dict)  # recursively call the function with each element
    elif hasattr(o, 'size'):  # if the object is not iterable, check if it has a 'size' attribute
        size = round(o.size)  # if yes, round its value and store it in the 'size' variable
        if size in size_dict:  # check if the 'size' value is already a key in the dictionary
            size_dict[size] += 1  # if yes, increment the value of the corresponding key by 1
        else:
            size_dict[size] = 1  # if not, create a new key-value pair with the 'size' as key and 1 as value

def show_ltitem_hierarchy(o: Any, depth=0):
    """Show location and text of LTItem and all its descendants"""
    if depth == 0:  # check if the function is being called for the first time (i.e., depth is 0)
        print('element                        fontname             text')  # if yes, print the header row
        print('------------------------------ -------------------- -----')

    print(
        f'{get_indented_name(o, depth):<30.30s} '  # print the indented class name
        f'{get_optional_fontinfo(o):<20.20s} '  # print the font info (if available)
        f'{get_optional_text(o)}'  # print the text (if available)
    )

    if isinstance(o, Iterable):  # check if the object is iterable
        for i in o:  # if yes, iterate over its elements
            show_ltitem_hierarchy(i, depth=depth + 1)  # recursively call the function with each element

path = Path('/content/CARE/sample_Eng.pdf').expanduser()  # create a Path object pointing to the PDF file
pages = extract_pages(path)  # extract the pages from the PDF

# Count the frequency of font sizes
size_dict = {}  # Create an empty dictionary to store font size frequencies
count_font_sizes(pages, size_dict)  # Call the function to count the font sizes

# Print the font size frequencies
print('Font size frequencies:')
print('---------------------')
for size, count in size_dict.items():  # Loop through the dictionary items
    print(f'{size}pt: {count}')  # Print the font size and its frequency


Font size frequencies:
---------------------
64pt: 5
44pt: 18
12pt: 32
11pt: 369
9pt: 22479
8pt: 390
10pt: 44
5pt: 4


Extract text and their font sizes

In [ ]:
from pdfminer.high_level import extract_pages
from pdfminer.layout import LTTextContainer, LTChar, LTLine, LAParams
import os

path = r'/content/CARE/sample_Eng.pdf'

Extract_Data = []

for page_layout in extract_pages(path):
    for element in page_layout:
        if isinstance(element, LTTextContainer):
            for text_line in element:
                if isinstance(text_line, LTTextContainer):
                    if len(text_line._objs) > 0:
                        first_char = text_line._objs[0]
                        if isinstance(first_char, LTChar):
                            font_size = first_char.size
                            text = element.get_text()
                            Extract_Data.append([font_size, text])

In [ ]:
with open("extracted_data2.txt", "w") as f:
    for data in Extract_Data:
        f.write("Font size: {}\n".format(data[0]))
        f.write("Text: {}\n".format(data[1]))
        f.write("\n")


**This extracts the font name, font_size, text, bounding box**

In [ ]:
from pdfminer.high_level import extract_pages        # Import function for extracting PDF pages
from pdfminer.layout import LTTextContainer, LTChar, LTLine, LAParams   # Import PDF layout classes
import os   # Import operating system functions

path = r'/content/CARE/sample_Eng.pdf'    # Define path to PDF file

output_file = 'efont_info.txt'   # Define name of output file

with open(output_file, 'w') as file:     # Open the output file for writing
    for page_layout in extract_pages(path):     # Iterate over PDF pages
        for element in page_layout:     # Iterate over elements on each page
            if isinstance(element, LTTextContainer):   # Check if element is a text container
                text = element.get_text()   # Get the text of the element
                bbox = element.bbox   # Get the bounding box of the element
                file.write(f"Text: {text}\n")   # Write text to output file
                file.write(f"Bounding Box: {bbox}\n")   # Write bounding box to output file
                for text_line in element:   # Iterate over lines of text in the container
                    if isinstance(text_line, LTTextContainer):   # Check if line is a text container
                        if len(text_line._objs) > 0:    # Check if line contains any objects
                            first_char = text_line._objs[0]   # Get the first object in the line
                            if isinstance(first_char, LTChar):   # Check if object is a character
                                font_size = first_char.size   # Get the font size of the character
                                font_name = first_char.fontname   # Get the font name of the character
                                file.write(f"Font size: {font_size}\n")   # Write font size to output file
                                file.write(f"Font name: {font_name}\n")   # Write font name to output file
                file.write("\n")   # Add newline to separate output for different elements


**This displays the bounding box, font and LT**

In [ ]:
from pdfminer.high_level import extract_pages
from pdfminer.layout import LTTextBox, LTTextLine, LTFigure, LTAnno

def get_optional_fontinfo(o):
    """Font info of LTChar if available, otherwise empty string"""
    if isinstance(o, LTChar):
        return f'{o.fontname} {round(o.size)}pt'
    return ''

def get_optional_text(o):
    """Text of LTItem if available, otherwise empty string"""
    if isinstance(o, LTTextBox) or isinstance(o, LTTextLine):
        return o.get_text().strip()
    return ''

def show_layout(objs, f):
    """
    Recursively parse layout objects and write information about each object's class name, bounding box,
    font size, font name, and text (if any) to a file.
    """
    for obj in objs:
        if isinstance(obj, LTTextBox):
            # Write the class name, bounding box, font size, font name, and text of the LTTextBox object to a file
            f.write(f'LTTextBox: {obj.bbox}, {get_optional_fontinfo(obj)}, {get_optional_text(obj)}\n')
        elif isinstance(obj, LTTextLine):
            # Write the class name, bounding box, font size, font name, and text of the LTTextLine object to a file
            f.write(f'LTTextLine: {obj.bbox}, {get_optional_fontinfo(obj)}, {get_optional_text(obj)}\n')
        elif isinstance(obj, LTFigure):
            # Recursively parse the layout objects of the LTFigure object
            show_layout(obj._objs, f)
        elif hasattr(obj, '_objs'):
            # Recursively parse the layout objects of the object
            show_layout(obj._objs, f)
        elif hasattr(obj, 'bbox'):
            # Write the class name and bounding box of the object to a file
            f.write(f'{obj.__class__.__name__}: {obj.bbox}\n')
        else:
            # Write the class name of the object to a file
            f.write(f'Class name: {obj.__class__.__name__}\n')


# Extract pages from PDF file
pages = extract_pages('/content/CARE/sample_Eng.pdf')

# Parse layout information for each page
with open('output.txt', 'w') as f:
    for i, page_layout in enumerate(pages, start=1):
        f.write(f'Page {i}:\n')
        show_layout(page_layout._objs, f)
        f.write('\n') # line separator for each page


In [ ]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')

def get_depth(paragraph):
    sentences = nltk.sent_tokenize(paragraph)
    tokens = [nltk.word_tokenize(sentence) for sentence in sentences]
    tagged = [nltk.pos_tag(token) for token in tokens]
    chunked = nltk.ne_chunk_sents(tagged, binary=True)
    depth = 0
    for tree in chunked:
        depth += tree.height()
    return depth

with open('/content/Dextracts/page3_info.txt', 'r') as file:
    text = file.read()
    paragraphs = text.split('\n\n')
    for i, paragraph in enumerate(paragraphs):
        depth = get_depth(paragraph)
        print(f"Depth of paragraph {i}: {depth}")
        # print("Paragraph text:")
        print(paragraph)
        print("--------------------")


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping chunkers/maxent_ne_chunker.zip.
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


Depth of paragraph 0: 3
Font size: 11.0
Font name: NDJLVI+ArialNarrow-Bold
Font color: (0.75, 0.05, 1, 0)
Text: REPORT TO SHAREHOLDERS 
--------------------
Depth of paragraph 1: 12
Font size: 9.0
Font name: FRIPNY+ArialNarrow
Font color: (0, 0, 0, 1)
Text: 2018  marked  a  change  in  food  and  even  drug  retail  with  the  absence  of  growth  or  expansion  at  less  than  population  growth  or  inflation  in  North 
America.  The net income attributable to equity holders of the Company attained $108.9 million, better than planned, yet shy of last year’s $119.3 million, 
while revenue grew slightly to $889.6 million.  This achievement was a blend of revenue growth in the modified atmosphere packaging, lidding and 
machinery product groups while our rigid container business receded as a result of transitioning to less costly plastic materials in our largest rigid container 
applications.  However, with the lack of retail growth and competitive selling price pressures intensifying 

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')

def get_depth(paragraph):
    sentences = nltk.sent_tokenize(paragraph)
    tokens = [nltk.word_tokenize(sentence) for sentence in sentences]
    tagged = [nltk.pos_tag(token) for token in tokens]
    chunked = nltk.ne_chunk_sents(tagged, binary=True)
    depth = 0
    for tree in chunked:
        depth += tree.height()
    return depth

output_file = "/content/CARE/Doutput.txt"  # Specify the output file path

with open('/content/Dextracts/page3_info.txt', 'r') as file:
    text = file.read()
    paragraphs = text.split('\n\n')

    with open(output_file, 'w') as output:
        for i, paragraph in enumerate(paragraphs):
            depth = get_depth(paragraph)
            output.write(f"Depth of paragraph {i}: {depth}\n")
            output.write(paragraph)
            output.write("\n\n")
            # output.write("--------------------\n")
            

print(f"Output saved to {output_file}")


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!


Output saved to /content/CARE/Doutput.txt


In [ ]:
import os
import csv

def convert_files_to_csv(input_folder, output_folder):
    os.makedirs(output_folder, exist_ok=True)

    for filename in os.listdir(input_folder):
        if filename.endswith('.txt'):
            input_file = os.path.join(input_folder, filename)
            output_file = os.path.join(output_folder, f'{os.path.splitext(filename)[0]}.csv')

            with open(input_file, 'r') as file:
                text = file.read()

            rows = []
            current_row = {}
            current_depth = 0

            for line in text.split('\n'):
                line = line.strip()

                if line.startswith('Font size:'):
                    current_row['Font size'] = line.split(':')[1].strip()
                elif line.startswith('Font name:'):
                    current_row['Font name'] = line.split(':')[1].strip()
                elif line.startswith('Font color:'):
                    current_row['Font color'] = line.split(':')[1].strip()
                elif line.startswith('Text:'):
                    current_row['Text'] = line.split(':')[1].strip()
                    rows.append(current_row)
                    current_row = {}
                    current_depth = 0
                else:
                    # Calculate the depth based on indentation level
                    depth = len(line) - len(line.lstrip())

                    # Update the depth if it has increased
                    if depth > current_depth:
                        current_depth = depth
                    # If the depth has decreased, consider it as the end of the current paragraph
                    elif depth < current_depth:
                        rows[-1]['Text'] += '\n' + line.strip()

            fieldnames = ['Font name', 'Font color', 'Font size', 'Text']

            with open(output_file, 'w', newline='') as file:
                writer = csv.DictWriter(file, fieldnames=fieldnames)
                writer.writeheader()
                writer.writerows(rows)

            print(f"Converted '{filename}' to '{output_file}'.")

input_folder = '/content/Dextracts'
output_folder = 'OUTput_folder'

convert_files_to_csv(input_folder, output_folder)


Converted 'page1_info.txt' to 'OUTput_folder/page1_info.csv'.
Converted 'page6_info.txt' to 'OUTput_folder/page6_info.csv'.
Converted 'page3_info.txt' to 'OUTput_folder/page3_info.csv'.
Converted 'page4_info.txt' to 'OUTput_folder/page4_info.csv'.
Converted 'page5_info.txt' to 'OUTput_folder/page5_info.csv'.
Converted 'page8_info.txt' to 'OUTput_folder/page8_info.csv'.
Converted 'page2_info.txt' to 'OUTput_folder/page2_info.csv'.
Converted 'page7_info.txt' to 'OUTput_folder/page7_info.csv'.
